In [1]:
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

import django
django.setup()

In [2]:
from data.models import HourData

In [3]:
import requests
from datetime import timedelta, datetime

In [4]:
up_dn_list = []
last_data = []
dn_list = []
up_list = []
new_dn_list = []
new_up_list = []
other_up_dn_list = []

continue_up_down = "D0"
prev_up_down = "S"
work_1_0_1 = "S"
work_1_5_1 = "S"
work_1_0_2 = "S"
work_1_5_2 = "S"
last_time_1_0_1 = datetime.now()
last_time_1_5_1 = datetime.now()
last_time_1_0_2 = datetime.now()
last_time_1_5_2 = datetime.now()

last_dn, last_up = "", ""

max_price, min_price = 0, 0
work_up_high, work_dn_low = 1000000, 0
volume = 0
prev_low = 0
prev_high = 100000
start = 0
last_up = 0
last_dn = 0
volume_up_dn = ""
hour_up_down = ""
prev_signal_up_price = 0
prev_signal_down_price = 0

In [5]:

while True:
    URL = "https://www.bitmex.com/api/v1/trade/bucketed?symbol=XBT&binSize=1h&partial=false&start={}&count=1000&reverse=false".format(start)
    req = requests.get(URL).json()
    for data in req:
        volume_up_dn = ""
        signal = ""
        signal_price = 0
        hour_up_down = ""
        now_work_1_0_1 = ""
        now_work_1_5_1 = ""
        now_work_1_0_2 = ""
        now_work_1_5_2 = ""
        datetime = datetime.strptime(data['timestamp'].replace("T", " ")[0:19], "%Y-%m-%d %H:%M:%S") + timedelta(hours=8)
        open_price = data['open'] if data['open'] != None else 0
        high_price = data['high'] if data['high'] != None else 0
        low_price = data['low'] if data['low'] != None else 0
        close_price = data['close'] if data['close'] != None else 0
        avg_price = (high_price + low_price) / 2
        
        if volume != 0:
            volume_rate = data['volume'] / volume
        else:
            volume_rate = 0
            
        volume = data['volume']

        flag = False

        if open_price > close_price:
            UD = "D"

        elif open_price < close_price:
            UD = "U"

        else:
            UD = "S"

        if len(last_data) > 3:
            last_data.pop(0)
            volume_bigger_count = 0
            volume_smaller_count = 0

            for i in last_data:
                if i[2] < volume:
                    volume_bigger_count += 1

                if i[2] > volume:
                    volume_smaller_count += 1

            if volume_bigger_count == 3:
                volume_up_dn = "UP"

            if volume_smaller_count == 3:
                volume_up_dn = "DN"

        last_data.append([low_price, high_price, volume])
        
        if continue_up_down[0] == "U":
            if UD == "U":
                continue_up_down = "U" + str(int(continue_up_down[1:]) + 1)
            elif UD == "D":
                continue_up_down = "D1"
        else:
            if UD == "U":
                continue_up_down = "U1"
            elif UD == "D":
                continue_up_down = "D" + str(int(continue_up_down[1:]) + 1)
        
        if len(up_dn_list) > 3:
            up_dn_list.pop(0)
        up_dn_list.append(UD)
        
        if up_dn_list.count("U") > 2:
            if high_price >= last_up:
                hour_up_down = "UP(U)"
            else:
                hour_up_down = "UP(D)"
            last_up = high_price
        
        if up_dn_list.count("D") > 2:
            if low_price >= last_dn:
                hour_up_down = "DN(U)"
            else:
                hour_up_down = "DN(D)"
            last_dn = low_price
            
        if UD == "U":
            if other_up_dn_list.count("D") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_U = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "D":
                        count += 1
                        max_price_list.append(dn_list[idx])
                    else:
                        is_U = True
                    if is_U and count >= 3:
                        if element == "U":
                            break
                        else:
                            min_price_list.append(up_list[idx])
                            break
                    min_price_list.append(up_list[idx])
                if prev_up_down == "D" and max(max_price_list) == max_price_list[len(max_price_list) - 1]:
                    if prev_signal_down_price != 0:
                        if prev_signal_down_price < min(min_price_list):
                            signal = "fD(U)"
                        elif prev_signal_down_price == min(min_price_list):
                            signal = "fD(0)"
                        else:
                            signal = "fD(D)"
                    prev_signal_down_price = min(min_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()

        if UD == "D":
            if other_up_dn_list.count("U") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_D = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "U":
                        count += 1
                        min_price_list.append(up_list[idx])
                    else:
                        is_D = True

                    if is_D and count >= 3:
                        if element == "D":
                            break
                        else:
                            max_price_list.append(dn_list[idx])
                            break
                    max_price_list.append(dn_list[idx])

                if prev_up_down == "U" and min(min_price_list) == min_price_list[len(min_price_list) - 1]:
                    print(datetime)
                    print(other_up_dn_list)
                    print(max_price_list, min_price_list)
                    if prev_signal_down_price != 0:
                        if prev_signal_up_price < max(max_price_list):
                            signal = "fU(U)"
                        elif prev_signal_up_price == max(max_price_list):
                            signal = "fU(0)"
                        else:
                            signal = "fU(D)"
                    prev_signal_up_price = max(max_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
        
        if len(dn_list) > 5:
            dn_list.pop(0)
        dn_list.append(high_price)
            
        if len(up_list) > 5:
            up_list.pop(0)
        up_list.append(low_price)
        
        if len(other_up_dn_list) > 5:
            other_up_dn_list.pop(0)
        other_up_dn_list.append(UD)
        
        if volume_up_dn == "UP":
            if (max_price < high_price) and (volume_rate > 1):
                if work_1_0_1[0] == "U":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0_1 = True
                    else:
                        work_1_0_1 = "U" + str(int(work_1_0_1[1:]) + 1)
                else:
                    work_1_0_1 = "U1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1):
                if work_1_0_1[0] == "D":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0 = True
                    else:
                        work_1_0_1 = "D" + str(int(work_1_0_1[1:]) + 1)
                
                else:
                    work_1_0_1 = "D1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
                
            if (max_price < high_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "U":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        continue_flag_1_5_1 = True
                    else:
                        work_1_5_1 = "U" + str(int(work_1_5_1[1:]) + 1)
                else:
                    work_1_5_1 = "U1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_1 = "D" + str(int(work_1_5_1[1:]) + 1)
                
                else:
                    work_1_5_1 = "D1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime

            if (max_price < high_price and min_price < low_price) and (volume_rate > 1):
                if work_1_0_2[0] == "U":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "U" + str(int(work_1_0_2[1:]) + 1)
                else:
                    work_1_0_2 = "U1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1):
                if work_1_0_2[0] == "D":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "D" + str(int(work_1_0_2[1:]) + 1)
                
                else:
                    work_1_0_2 = "D1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime     
                
                
            if (max_price < high_price and min_price < low_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "U":
                    if datetime == last_time_1_5_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "U" + str(int(work_1_5_2[1:]) + 1)
                else:
                    work_1_5_2 = "U1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "D" + str(int(work_1_5_2[1:]) + 1)
                
                else:
                    work_1_5_2 = "D1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime

            max_price, min_price = high_price, low_price
            
        if now_work_1_0_1 != "":
            now_work_1_0_1 = "w" + now_work_1_0_1
        
        if now_work_1_5_1 != "":
            now_work_1_5_1 = "w" + now_work_1_5_1
            
        if now_work_1_0_2 != "":
            now_work_1_0_2 = "w" + now_work_1_0_2
            
        if now_work_1_5_2 != "":
            now_work_1_5_2 = "w" + now_work_1_5_2
        
        print(hour_up_down)
        element = HourData()
        element.datetime = datetime
        element.open_price = open_price
        element.min_price = low_price
        element.max_price = high_price
        element.close_price = close_price
        element.avg_price = avg_price
        element.volume = volume
        element.volume_rate = volume_rate
        element.volume_up_dn = volume_up_dn
        element.up_down = UD
        element.continue_up_down = continue_up_down
        element.hour_up_down = hour_up_down
        element.signal = signal
        element.signal_price = signal_price
        element.work_two_1_0 = now_work_1_0_1
        element.work_two_1_5 = now_work_1_5_1
        element.work_one_1_0 = now_work_1_0_2
        element.work_one_1_5 = now_work_1_5_2
        element.save()
        
        prev_up_down = UD
        
    start += 1000




DN(U)




DN(D)
DN(D)
DN(U)

UP(U)
2015-09-26 10:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[235.29, 235.36, 235.2] [235.29, 235.19, 234.9]
UP(U)








DN(D)
DN(D)


UP(D)
2015-09-27 00:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[235, 234.35, 234.87, 233.85] [235, 234.1, 232.91]

DN(U)


UP(D)
UP(D)
2015-09-27 06:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[234.67, 234.73, 234, 233.8] [234.32, 234.05, 233.69, 233.68]
UP(D)
UP(D)












DN(D)
DN(U)
DN(D)





UP(D)


DN(U)
DN(D)
DN(D)
DN(U)

UP(U)
UP(D)
UP(D)
2015-09-28 15:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[237.8, 237.82, 238.23, 234.58, 233.93] [237.37, 237.34, 234.45, 233.6, 233.39]
UP(D)
UP(U)


UP(U)
2015-09-28 21:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[239.16, 238.6, 240.49, 238.7] [238.57, 238.31, 237.3]

UP(D)
UP(U)
UP(D)
2015-09-29 01:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[239.31, 239.7, 238.19] [238.8, 238.15, 237.87]
UP(D)


2015-09-29 04:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[239, 238.87, 239, 239.31, 239.7] [239, 238.8, 238.15]
DN(U)


2015-10-18 03:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[273.77, 274.8, 274.71, 274.7, 275] [271.74, 271.75, 270.5]

DN(D)
DN(U)
DN(U)

UP(U)
UP(U)
2015-10-18 10:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[275.05, 275, 274.57, 272.58] [273.47, 272.7, 270.62, 267.82]
UP(D)
UP(D)

DN(U)


UP(D)
2015-10-18 17:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[272.65, 272.52, 271.17] [270.76, 271.48, 269.08]
UP(D)

DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(U)


DN(U)

2015-10-19 07:00:00
['U', 'D', 'D', 'U', 'D', 'U']
[265.73, 266.05, 266.4, 266.9, 264.6, 265.21] [264.65, 264.15, 263.25]
DN(U)

UP(D)
UP(U)
2015-10-19 11:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[267.73, 266.99, 265.69] [266.38, 265.68, 263.96]
UP(U)
UP(D)
2015-10-19 13:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[267.49, 267.76, 267.73, 266.99] [266.65, 266.38, 265.68]

DN(D)
DN(D)
DN(U)




DN(D)


DN(U)


2015-10-20 03:00:00
['U', 'U', 'D', 'D', 'U', 'D']
[266.04, 266.12, 266.5, 266.8, 266.98] [265.25, 265.09, 263.33]

UP(D)

DN(D)

DN(U)

UP(U)
2015-10


DN(D)
DN(U)

DN(U)

UP(U)
UP(U)
UP(U)
UP(U)
2015-11-04 04:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[430, 429.03, 416, 404.91, 407.9] [416, 410.24, 402.77, 397.04, 388]
UP(U)
UP(U)


UP(D)
UP(U)
UP(U)
UP(U)
UP(U)
UP(U)
2015-11-04 14:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[465.73, 454.22, 447.07, 439.94, 429.98, 422] [433, 440.44, 434.69, 417.64, 416.89, 410.86]
UP(D)


DN(U)

UP(D)
UP(U)
UP(U)
UP(U)
2015-11-04 23:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[545.26, 526.33, 474, 464.53, 457] [520.71, 485, 460.36, 440.58, 437.48]
UP(U)
UP(D)
2015-11-05 01:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[542, 551.77, 545.26, 526.33] [510, 520.71, 485]

DN(U)




DN(D)
DN(U)
DN(U)

UP(D)
2015-11-05 12:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[463.5, 450, 427.08, 433.16] [444.27, 409.2, 390]


DN(D)
DN(U)
DN(U)
DN(D)
DN(U)
DN(U)
DN(U)

UP(D)
2015-11-05 23:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[415.7, 406.57, 398.3, 408] [396.59, 386.4, 384.4]



2015-11-06 02:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[409, 406.46, 414.19, 4

DN(U)
DN(U)
DN(D)






2015-11-22 14:00:00
['U', 'U', 'D', 'S', 'D', 'U']
[328.2, 327.75, 327.47, 328.6, 328.5, 328.28] [327.58, 327, 325.6]



DN(U)
DN(D)

DN(D)
DN(D)

DN(D)
DN(D)




DN(D)
DN(D)

UP(D)
2015-11-23 09:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[325.03, 324.22, 326.03] [322.73, 322.8, 320.27]
UP(D)


DN(U)
DN(D)



UP(D)



DN(U)


2015-11-24 00:00:00
['U', 'U', 'D', 'D', 'U', 'D']
[326.08, 323.81, 331.94, 324.74, 325.68] [323.54, 322.93, 322.19]



2015-11-24 03:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[324.28, 324.02, 325.44, 326.08, 323.81] [323.24, 323.54, 322.93]
DN(U)



2015-11-24 07:00:00
['U', 'U', 'S', 'D', 'U', 'D']
[325.82, 325.03, 323.5, 324.2, 324.28] [324.39, 323.6, 323.24]

UP(U)

DN(D)
DN(D)
DN(U)
DN(D)

UP(D)

UP(U)
2015-11-24 18:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[321.51, 321.8, 321.5, 321.23] [320.4, 320.92, 318.62]




2015-11-24 22:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[323.76, 323.1, 322.59, 321.22, 321.51] [322.24, 320.71, 320.4]

DN(U)




DN(D)
DN(U)



UP(D)
UP(D)
UP(U)
2015-12-10 20:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[416.83, 416, 416.29, 415.41] [415.37, 414.17, 412]

DN(U)

2015-12-10 23:00:00
['U', 'D', 'D', 'U', 'D', 'U']
[422.21, 416.8, 416.09, 416.83, 416, 416.29] [414.99, 415.37, 414.17]
DN(U)
DN(D)
DN(U)
DN(D)
DN(U)
DN(D)
DN(D)
DN(U)
DN(U)
DN(D)


UP(U)
UP(U)
UP(U)
2015-12-11 14:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[426.86, 426.43, 422.1, 423.43, 418.84] [424.64, 421.69, 420.79, 418.64, 416.64]
UP(U)

DN(U)
DN(U)
2015-12-11 18:00:00
['U', 'D', 'D', 'D', 'U', 'U']
[435, 423.7, 424.36, 426.9, 426.86, 426.43] [422.86, 424.64, 421.69]
DN(U)



2015-12-11 22:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[440, 438.39, 435.82, 435.36, 435] [434.83, 433.36, 422.86]

DN(D)


UP(U)
UP(U)
UP(U)
UP(U)
2015-12-12 06:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[463.78, 449.39, 447.03, 443.47, 439.6, 435.11] [443.7, 443, 439.65, 436.1, 434.87, 432.34]
UP(D)


2015-12-12 09:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[462.22, 454.76, 456.82, 463.78, 449.39] [4

...

DN(U)
DN(D)
DN(U)
DN(D)

UP(D)
2021-03-10 06:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[54592.5, 54220, 54058.5] [54073.5, 53821, 53660]
UP(D)
UP(U)
UP(U)
UP(U)
2021-03-10 10:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[55827, 54965, 54922] [54864.5, 54561, 54195.5]
UP(U)

DN(D)
DN(U)
DN(D)

UP(D)
UP(U)
UP(U)
2021-03-10 19:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[55479, 54750, 54422, 54099.5] [54340, 54100, 53920.5, 53583]
UP(D)
UP(D)
UP(U)
UP(U)
2021-03-10 23:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[56436.5, 55550, 55021] [55213, 54929.5, 54610]
UP(U)
UP(U)
UP(U)
2021-03-11 02:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[57200, 56600, 56476, 56436.5] [56195.5, 56067.5, 55213]


DN(U)
DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(D)



DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(U)

UP(D)
UP(U)
2021-03-11 23:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[57100, 56444, 56259, 55580] [56327.5, 55813, 55458.5, 54455]
UP(U)


2021-03-12 02:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[57060, 57192, 57316.5, 57100, 56444] [56370, 56327.5, 55813]
DN(U)

20

DN(D)
DN(U)
DN(U)

UP(U)
2021-03-28 02:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[56251, 55051, 54780, 54878] [54970.5, 54541, 54127]




UP(U)
2021-03-28 07:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[56682.5, 55942, 55914] [55766, 55734, 55688]
UP(D)




UP(D)
UP(U)






DN(U)
DN(D)
DN(U)

UP(D)
2021-03-29 01:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[56398, 55880, 55889] [55720, 55540, 55509]
UP(U)

DN(D)
DN(D)
DN(U)
DN(U)

UP(D)
2021-03-29 09:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[55878.5, 55670, 55371] [55550, 55064, 54862.5]
UP(U)


2021-03-29 12:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[55509.5, 55496.5, 55928.5, 55878.5, 55670] [55186, 55550, 55064]
DN(U)
DN(D)
DN(D)
DN(U)

UP(U)
UP(U)
UP(U)
2021-03-29 20:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[58400, 58323, 57300, 56180, 55501] [57911.5, 57093.5, 55970.5, 55388.5, 55081]
UP(D)
UP(U)
2021-03-29 22:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[58490, 58156.5, 58400, 58323] [57887, 57911.5, 57093.5]



DN(U)

DN(U)
DN(D)
DN(D)
DN(U)



DN(U)
DN(D)
DN(D)
DN(D)

DN(U)
DN(D)
DN(D)
DN(D)


UP(D)
UP(U)
UP(U)
2021-04-17 02:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[61872, 61765.5, 61552] [61377.5, 61293, 60647]
UP(U)
UP(U)



DN(U)
DN(D)
DN(D)

UP(U)
UP(U)
2021-04-17 13:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[62598, 62350, 62050, 61764.5] [62197.5, 61938.5, 61519.5, 61240]
UP(D)

DN(U)
DN(U)



DN(D)
DN(D)




DN(D)

UP(D)
UP(U)
2021-04-18 06:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[60942, 60833, 60550] [60694.5, 60424.5, 59673]
UP(U)

DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)

UP(D)
UP(U)
2021-04-18 17:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[56638, 56509.5, 55949, 56069] [55912.5, 55520, 54900, 54166]
UP(D)

DN(U)
DN(D)
DN(U)


UP(D)
2021-04-19 01:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[55740, 55927, 55405.5, 55050] [54890, 54780.5, 53375]

DN(U)


UP(U)
UP(U)
UP(U)
2021-04-19 08:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[56736.5, 56460, 56340, 55897.5, 55788.5] [56140, 55706.5, 55819.5, 55417, 55112]
UP(D)
UP(U)
2021-04-19 10:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[56899,



2021-08-08 19:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[44940, 45050, 45320, 45339, 44869] [44487.5, 44682, 44235]
DN(D)

UP(D)



DN(D)
DN(D)
DN(D)
DN(D)

UP(D)
2021-08-09 07:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[44239, 44264, 43875] [43793, 43597, 43302]
UP(U)

DN(D)
DN(D)
DN(U)

UP(D)
2021-08-09 14:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[43610, 43631.5, 43499.5] [43278.5, 43299.5, 43084]
UP(U)


DN(U)

UP(U)
UP(U)
2021-08-09 21:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[46088.5, 45717.5, 45058] [45520, 44820.5, 43688]
UP(D)






2021-08-10 04:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[46489.5, 46081, 46207.5, 46069, 46320] [45872.5, 45664.5, 45431.5]
DN(U)
DN(D)


UP(U)
2021-08-10 09:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[46420, 46346, 46052] [46076.5, 45694.5, 44926.5]
UP(U)
UP(D)
UP(U)



DN(D)

UP(U)






DN(D)
DN(D)
DN(U)

UP(D)
UP(U)
UP(U)
2021-08-11 07:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[45934, 45774.5, 45570, 45364.5, 45110.5] [45472, 45415, 45230, 45010, 44780.5]
UP(D)
UP(D)
UP(U)
UP(U

TypeError: string indices must be integers